In [2]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score
from helpers import *
# Load the TSV file
df = pd.read_csv('ml-32m/ratings.csv')


# Print the head of the DataFrame
print(df.head())

2024-11-27 16:09:37.109993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40


In [4]:
data_folder = './MovieSummaries/'
#paths to files
plot_summaries_path = data_folder + 'plot_summaries.txt'
movie_metadata_path = data_folder + 'movie.metadata.tsv'
character_metadata_path = data_folder + 'character.metadata.tsv'

# load the data
# 1. Plot summaries data
plot_summaries_df = pd.read_csv(plot_summaries_path, delimiter='\t', names=['wikipedia_movie_id', 'plot_summary'], 
                                 encoding='utf-8')

# 2. Movie metadata
movie_metadata_df = pd.read_csv(movie_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 
                                                                            'movie_name', 'release_date', 'box_office_revenue',
                                                                            'runtime', 'languages', 'countries', 'genres'], 
                                 encoding='utf-8')

# 3. Character metadata
character_metadata_df = pd.read_csv(character_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 'release_date', 'character_name', 
                                                                                    'actor_dob', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name', 
                                                                                    'actor_age_at_release', 'freebase_character_actor_map_id', 'freebase_character_id', 
                                                                                    'freebase_actor_id'], 
                                     encoding='utf-8')


In [5]:
movies = pd.read_csv('ml-32m/movies.csv')

In [6]:
movie_metadata_df['movie_name_formatted'] = movie_metadata_df['movie_name'].str.lower().str.strip()
movies['title_format'] = movies['title'].str[:-6].str.strip().str.lower()


In [7]:
movies.head()

,movieId,title,genres,title_format
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii


In [8]:
# calculate the number of common movies between the two datasets

common_movies = set(movie_metadata_df['movie_name_formatted']).intersection(set(movies['title_format']))
print('Number of common movies:', len(common_movies))

# merge the two datasets

merged_df = pd.merge(movies, movie_metadata_df, left_on='title_format', right_on='movie_name_formatted', how='inner')

Number of common movies: 12318


In [9]:
merged_df

,movieId,title,genres_x,title_format,wikipedia_movie_id,freebase_movie_id,movie_name,release_date,box_office_revenue,runtime,languages,countries,genres_y,movie_name_formatted
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,53085,/m/0dyb1,Toy Story,1995-11-19,361958736.0,77.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0556j8"": ""Buddy film"", ""/m/03k9fj"": ""Adve...",toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji,3700174,/m/09w353,Jumanji,1995-12-15,262797249.0,104.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/0hj3n2k"": ""Fanta...",jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men,1934035,/m/0676dr,Grumpier Old Men,1995-12-22,71518503.0,101.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0556j8"": ""...",grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale,972970,/m/03vny7,Waiting to Exhale,1995-12-22,81452156.0,121.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3n0w"": ""Ensemble Film"", ""/m/06w2n3t"": ...",waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii,3303622,/m/094g2z,Father of the Bride Part II,1995-12-08,76594107.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06qm3"": ""Screwball comedy"", ""/m/02l7c8"": ...",father of the bride part ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,131152,The Fat Spy (1966),Comedy,the fat spy,1853200,/m/0613sx,The Fat Spy,1966,NaN,75.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0gf28"": ""Parody"", ""/m/0220p9g"": ""Musical ...",the fat spy
17558,131166,WWII IN HD (2009),(no genres listed),wwii in hd,26321497,/m/0bbxzy2,WWII in HD,NaN,NaN,NaN,{},{},"{""/m/082gq"": ""War film""}",wwii in hd
17559,131239,Three Quarter Moon (2011),Comedy|Drama,three quarter moon,33597212,/m/0hgrd5g,Three Quarter Moon,2011-09-30,NaN,95.0,"{""/m/02hwyss"": ""Turkish Language"", ""/m/0gtg"": ...","{""/m/0345h"": ""Germany""}","{""/m/0556j8"": ""Buddy film"", ""/m/0hqxf"": ""Famil...",three quarter moon
17560,131248,Brother Bear 2 (2006),Adventure|Animation|Children|Comedy|Fantasy,brother bear 2,4118248,/m/0bk2k2,Brother Bear 2,2006-08-29,NaN,73.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hqxf"": ""Family Film"", ""/m/0hcr"": ""Animat...",brother bear 2


In [10]:
# Remove this user from the dataset, because it has too many ratings
df = df[df['userId'] != 175325]

In [11]:
list_of_genres=movies['genres'].unique()

In [12]:
# Print the user with the most ratings
user_ratings_count = df['userId'].value_counts()

print('User with the most ratings:')
print(user_ratings_count.head(20))


User with the most ratings:
118205    9254
8405      7515
82418     5646
121535    5520
125794    5491
74142     5447
34576     5356
131904    5330
83090     5169
59477     4988
130767    4785
79159     4707
8963      4524
15617     4354
92011     4236
71975     4182
20132     4101
46470     4094
88820     4093
63147     3958
Name: userId, dtype: int64


In [13]:
rating_films=pd.merge(movies,df,on='movieId',how='inner')


In [14]:
unique_genres = movies['genres'].unique()
all_genres = set('|'.join(unique_genres).split('|'))

all_genres_list = list(all_genres)
all_genres_list.remove('(no genres listed)')
print(all_genres_list)


['IMAX', 'Crime', 'Mystery', 'Sci-Fi', 'Musical', 'Comedy', 'Children', 'Adventure', 'Film-Noir', 'Thriller', 'Animation', 'Action', 'Western', 'Documentary', 'War', 'Horror', 'Drama', 'Fantasy', 'Romance']


In [15]:
# Create a one hot encoding for each genre
for genre in all_genres_list : 
    rating_films[genre] = rating_films['genres'].apply(lambda x: genre in x.split('|'))
rating_films

,movieId,title,genres,title_format,userId,rating,timestamp,IMAX,Crime,Mystery,...,Thriller,Animation,Action,Western,Documentary,War,Horror,Drama,Fantasy,Romance
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,3,4.0,1999-12-11 13:36:47,False,False,False,...,False,True,False,False,False,False,False,False,True,False
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,6,5.0,1997-03-13 17:50:52,False,False,False,...,False,True,False,False,False,False,False,False,True,False
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,8,4.0,1996-06-05 13:37:51,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,10,4.0,1999-11-25 02:44:47,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,11,4.5,2009-01-02 01:13:41,False,False,False,...,False,True,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000258,131254,Kein Bund für's Leben (2007),Comedy,kein bund für's leben,79570,4.0,2015-03-30 19:32:59,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20000259,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,"feuer, eis & dosenbier",79570,4.0,2015-03-30 19:48:08,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20000260,131258,The Pirates (2014),Adventure,the pirates,28906,2.5,2015-03-30 19:56:32,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20000261,131260,Rentun Ruusu (2001),(no genres listed),rentun ruusu,65409,3.0,2015-03-30 19:57:46,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
def rating_for_userID_generator(userId):
    user_rating = rating_films[rating_films['userId'] == userId]
    user_rating = user_rating.iloc[:, 5:].sample(frac=1)  
    user_rating = user_rating.drop(columns=['timestamp'])
    y = user_rating['rating']
    x = user_rating.drop(columns=['rating'])
    
    length = len(y)
    x_size = int(length * 0.8) 
    
    x_train = x.iloc[:x_size, :]
    x_test = x.iloc[x_size:, :]
    y_train = y.iloc[:x_size]
    y_test = y.iloc[x_size:]
    
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = rating_for_userID_generator(8405)


In [17]:


# Define the neural network model
model = Sequential([
    Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Predict on both training and test sets
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Evaluate the model
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Training Set Evaluation:")
print("MSE:", train_mse)
print("R^2 Score:", train_r2)

print("\nTest Set Evaluation:")
print("MSE:", test_mse)
print("R^2 Score:", test_r2)



Epoch 1/100
151/151 [==============================] - 2s 4ms/step - loss: 2.2966 - mse: 2.2966 - val_loss: 0.8898 - val_mse: 0.8898
Epoch 2/100
151/151 [==============================] - 0s 2ms/step - loss: 0.8343 - mse: 0.8343 - val_loss: 0.8070 - val_mse: 0.8070
Epoch 3/100
151/151 [==============================] - 0s 3ms/step - loss: 0.7923 - mse: 0.7923 - val_loss: 0.7792 - val_mse: 0.7792
Epoch 4/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7807 - mse: 0.7807 - val_loss: 0.7674 - val_mse: 0.7674
Epoch 5/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7691 - mse: 0.7691 - val_loss: 0.7682 - val_mse: 0.7682
Epoch 6/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7654 - mse: 0.7654 - val_loss: 0.7734 - val_mse: 0.7734
Epoch 7/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7606 - mse: 0.7606 - val_loss: 0.7863 - val_mse: 0.7863
Epoch 8/100
151/151 [==============================] - 0s 2ms/step - 

In [18]:


def evaluate_model_for_user(userId, data):
    """
    Generates training and test splits for a given user ID, trains a neural network, 
    and evaluates the model to return MSE and R^2 for both training and test sets.
    """
    # Filter and preprocess data for the given user
    user_rating = data[data['userId'] == userId]
    user_rating = user_rating.iloc[:, 5:].sample(frac=1)  # Shuffle data
    user_rating = user_rating.drop(columns=['timestamp'])
    y = user_rating['rating']
    x = user_rating.drop(columns=['rating'])

    # Split into training and testing sets
    length = len(y)
    x_size = int(length * 0.8)
    x_train, x_test = x.iloc[:x_size, :], x.iloc[x_size:, :]
    y_train, y_test = y.iloc[:x_size], y.iloc[x_size:]

    # Define the neural network model
    model = Sequential([
        Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
        Dense(32, activation='relu'),  # Second hidden layer
        Dense(1, activation='linear')  # Output layer for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Train the model
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

    # Predict on training and test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Calculate evaluation metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Print evaluation results
    print(f"Evaluation for User ID {userId}:")
    print("Training Set Evaluation:")
    print(f"MSE: {train_mse:.4f}, R^2: {train_r2:.4f}")
    print("\nTest Set Evaluation:")
    print(f"MSE: {test_mse:.4f}, R^2: {test_r2:.4f}")

    # Return MSE values for both sets
    return train_mse, test_mse

train_mse, test_mse = evaluate_model_for_user(8405, rating_films)


Epoch 1/100
151/151 [==============================] - 1s 3ms/step - loss: 2.7443 - mse: 2.7443 - val_loss: 0.8438 - val_mse: 0.8438
Epoch 2/100
151/151 [==============================] - 0s 2ms/step - loss: 0.8279 - mse: 0.8279 - val_loss: 0.7574 - val_mse: 0.7574
Epoch 3/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7862 - mse: 0.7862 - val_loss: 0.7422 - val_mse: 0.7422
Epoch 4/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7678 - mse: 0.7678 - val_loss: 0.7473 - val_mse: 0.7473
Epoch 5/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7630 - mse: 0.7630 - val_loss: 0.7568 - val_mse: 0.7568
Epoch 6/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7605 - mse: 0.7605 - val_loss: 0.7484 - val_mse: 0.7484
Epoch 7/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7554 - mse: 0.7554 - val_loss: 0.7423 - val_mse: 0.7423
Epoch 8/100
151/151 [==============================] - 0s 2ms/step - 

In [19]:
ratings_model_df=one_hot_encoding_genre(df,movies)

In [21]:
Total_eval(8405,df,movies)

KeyError: 'director'

## Adding data 

In [22]:
df_tmdb_movie = pd.read_csv('TMdB_Movielens_combined/movies_metadata.csv', low_memory=False)
df_tmdb_credits = pd.read_csv('TMdB_Movielens_combined/credits.csv')
df_tmdb_keywords = pd.read_csv('TMdB_Movielens_combined/keywords.csv')

In [23]:
#director, length, vote_average, release_date
df_tmdb_keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [24]:
df_tmdb_movie.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
df_tmdb_credits.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
df_tmdb_keywords.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
# To merge we need to have a common type 
df_tmdb_movie['id'] = pd.to_numeric(df_tmdb_movie['id'], errors='coerce')
df_tmdb_credits['id'] = pd.to_numeric(df_tmdb_credits['id'], errors='coerce')
df_tmdb_keywords['id'] = pd.to_numeric(df_tmdb_keywords['id'], errors='coerce')
df_tmdb_movie.dropna(subset=['id'], inplace=True)
df_tmdb_credits.dropna(subset=['id'], inplace=True)
df_tmdb_keywords.dropna(subset=['id'], inplace=True)
df_tmp = pd.merge(df_tmdb_movie, df_tmdb_credits, on='id', how='inner')
df_tmp = df_tmp.rename(columns={"original_title": "title_format"})
df_tmp['title_format']=df_tmp['title_format'].apply(
    lambda x : x.lower()
)
df_tmp.head(2)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,title_format,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,toy story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [25]:
selected_columns = df_tmp[['title_format', 'runtime', 'crew']]
selected_columns['director'] = selected_columns['crew'].apply(get_director)
selected_columns.drop(columns='crew', inplace=True)
test_merge=pd.merge(selected_columns,movies, on='title_format',how='inner')
test_merge[test_merge['director'].isna()]='nobody'
director_counts = test_merge['director'].value_counts()
test_merge['director'] = test_merge['director'].apply(
    lambda x: x if director_counts[x] > 10 else 'nobody'
)
test_merge.drop(columns=['title_format','title'], inplace=True)
test_merge

/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_69795/3334071302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['director'] = selected_columns['crew'].apply(get_director)
/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_69795/3334071302.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.drop(columns='crew', inplace=True)


,runtime,director,movieId,genres
0,81.0,nobody,1,Adventure|Animation|Children|Comedy|Fantasy
1,104.0,nobody,2,Adventure|Children|Fantasy
2,101.0,nobody,3,Comedy|Romance
3,127.0,nobody,4,Comedy|Drama|Romance
4,106.0,nobody,5,Comedy
...,...,...,...,...
18617,705.0,nobody,7502,Action|Drama|War
18618,60.0,nobody,27627,Drama|Romance
18619,89.0,nobody,121821,Drama|Romance
18620,143.0,nobody,96704,Drama


In [26]:
def Total_eval_test(userId,df,movies) :
    """
    Creates one hot encoded features per unique genre 

    """
    # Merges df and movies
    mini_df=df[df['userId']==userId]
    rating_films=pd.merge(movies, mini_df, on='movieId', how='inner')
    rating_films.drop(columns='movieId',inplace=True)
    # Create list of unique_genres
    unique_genres = movies['genres'].unique()
    all_genres = set('|'.join(unique_genres).split('|'))

    all_genres_list = list(all_genres)
    all_genres_list.remove('(no genres listed)')
    
    unique_director = movies['director'].unique()

    all_director_list = list(unique_director)
    all_director_list.remove('nobody')
    # Create a one hot encoding for each genre
    for genre in all_genres_list:
        rating_films[genre] = rating_films['genres'].apply(lambda x: genre in x.split('|'))
    # Create a one hot encoding for each director
    for director in all_director_list:
        rating_films[director] = rating_films['director'].apply(lambda x: director in x)
    rating_films.drop(columns=['director','genres','timestamp'],inplace=True)
    return rating_films

In [32]:
testing=Total_eval_test(200,df,test_merge)

/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_69795/1370888562.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rating_films[director] = rating_films['director'].apply(lambda x: director in x)
/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_69795/1370888562.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rating_films[director] = rating_films['director'].apply(lambda x: director in x)
/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_69795/1370888562.py:26: PerformanceWarning: DataFrame i

In [35]:
def evaluate_model_for_user_test(userId, data):
    """
    Generates training and test splits for a given user ID, trains a neural network,
    and evaluates the model to return MSE and R^2 for both training and test sets.
    """
    # Filter and preprocess data for the given user
    user_rating = data[data['userId'] == userId]
    user_rating = user_rating.sample(frac=1)  # Shuffle data
    y = user_rating['rating'].astype('float32')
    x = user_rating.drop(columns='rating').astype('float32')

    # Split into training and testing sets
    length = len(y)
    x_size = int(length * 0.8)
    x_train, x_test = x.iloc[:x_size, :], x.iloc[x_size:, :]
    y_train, y_test = y.iloc[:x_size], y.iloc[x_size:]

    # Define the neural network model
    model = Sequential([
        Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
        Dense(32, activation='relu'),  # Second hidden layer
        Dense(1, activation='linear')  # Output layer for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Train the model
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.3, verbose=1)

    # Predict on training and test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Calculate evaluation metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Print evaluation results
    print(f"Evaluation for User ID {userId}:")
    print("Training Set Evaluation:")
    print(f"MSE: {train_mse:.4f}, R^2: {train_r2:.4f}")
    print("\nTest Set Evaluation:")
    print(f"MSE: {test_mse:.4f}, R^2: {test_r2:.4f}")

    # Return MSE values for both sets
    return train_mse, test_mse


In [36]:
evaluate_model_for_user_test(200,testing)

Epoch 1/100
2/2 [==============================] - 1s 217ms/step - loss: 18.7990 - mse: 18.7990 - val_loss: 7.5552 - val_mse: 7.5552
Epoch 2/100
2/2 [==============================] - 0s 39ms/step - loss: 7.9480 - mse: 7.9480 - val_loss: 12.3506 - val_mse: 12.3506
Epoch 3/100
2/2 [==============================] - 0s 82ms/step - loss: 9.7761 - mse: 9.7761 - val_loss: 1.1373 - val_mse: 1.1373
Epoch 4/100
2/2 [==============================] - 0s 108ms/step - loss: 1.4080 - mse: 1.4080 - val_loss: 3.4169 - val_mse: 3.4169
Epoch 5/100
2/2 [==============================] - 0s 97ms/step - loss: 4.3572 - mse: 4.3572 - val_loss: 4.1603 - val_mse: 4.1603
Epoch 6/100
2/2 [==============================] - 0s 108ms/step - loss: 4.3740 - mse: 4.3740 - val_loss: 0.7248 - val_mse: 0.7248
Epoch 7/100
2/2 [==============================] - 0s 40ms/step - loss: 1.1648 - mse: 1.1648 - val_loss: 1.9144 - val_mse: 1.9144
Epoch 8/100
2/2 [==============================] - 0s 35ms/step - loss: 1.9887 - ms

(0.31638274, 1.2003912)

In [37]:
a=df_tmdb_keywords[df_tmdb_keywords['id']==862]
a

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [41]:
micro_df=df.drop(columns='timestamp')
micro_df.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [42]:
def gradient(m, u, r, delta_m, delta_u):
    error = np.dot(m, u.T) - r 
    grad_m = 2 * error * u + 2 * delta_m * m  
    grad_u = 2 * error * m + 2 * delta_u * u  
    return grad_m, grad_u
class Predictor:
    def __init__(self, train_data):
        # This is where you can do any preparation or 'training' if necessary.
        pass

    def __call__(self, test_data):
        """
        Make predictions for the users and movies in the test dataset
        (without looking at the ratings).

        Inputs:
            test_data: Dataset (with `n` entries)

        Output:
            predictions: np.array of floats, shape (`n`)
        """
        raise NotImplementedError()
    
class MatrixFactorizationPredictor(Predictor):
    def __init__(self, train_data, num_features=20, seed=1):
        # Randomly initialize features for the users and the movies from N(0, 1)

        # use this generator (https://numpy.org/doc/stable/reference/random/index.html)
        # you are expected to use rng.normal() twice in this function to match the tests, once for movies, and then once for users
        rng = np.random.default_rng(seed)

        num_movies = np.max(train_data.movies) + 1
        num_users = np.max(train_data.users) + 1
        ####################################
        ### ___ ENTER YOUR CODE HERE ___ ###
        ####################################
        
        self.movies_=train_data['movie'].to_numpy()
        self.users_=train_data['user'].to_numpy()
        self.ratings_=train_data['rating'].to_numpy()
        self.movie_features = rng.normal(loc=0.0, scale=1.0, size=(num_movies, num_features))
        self.user_features = rng.normal(loc=0.0, scale=1.0, size=(num_users, num_features))
        
        # Normally, you should train the model here, but we will skip this
        # for now, to be able to take it step-by-step.
    def gradient(m, u, r, delta_m, delta_u):
        error = np.dot(m, u.T) - r 
        grad_m = 2 * error * u + 2 * delta_m * m  
        grad_u = 2 * error * m + 2 * delta_u * u  
        return grad_m, grad_u
    def train(self, movie,user,rating, weight_decay_movie=0.3,weight_decay_user=0.3,learning_rate=0.02):
        """
        Train the model using Stochastic Gradient Descent.
        """
         
        movie_feature_vector = self.movie_features[movie, :]
        user_feature_vector = self.user_features[user, :]
                
        grad_m, grad_u = gradient(movie_feature_vector, user_feature_vector, rating, weight_decay_movie, weight_decay_user)
                    
        # Update the feature vectors
        self.movie_features[movie, :] -= learning_rate * grad_m
        self.user_features[user, :] -= learning_rate * grad_u      
    def train_user(self,user, weight_decay_user=0.3, verbose = False):
        """
        Train the model using Stochastic Gradient Descent.
        """
        mask = self.users_ == user
        user_movies = self.movies_[mask]
        ratings=self.ratings_[mask]
        M = self.movie_features[user_movies]
        A = M.T @ M + weight_decay_user * np.eye(M.shape[1])  # Matrix for solving
        b = M.T @ ratings
        if verbose:
            print("Avant" )
            print(self.user_features[user][0] )
            print()

        self.user_features[user] = np.linalg.solve(A, b)
        if verbose:
            print("Après" )
            print(self.user_features[user][0] )
            print()
        
    def train_movies(self, movie, weight_decay_movie=0.3, verbose=True):
        """
        Train the model using Stochastic Gradient Descent.
        """
         
        mask = self.movies_ == movie
        movie_users = self.users_[mask]
        ratings=self.ratings_[mask]

        U = self.user_features[movie_users]
        A = U.T @ U + weight_decay_movie * np.eye(U.shape[1])  # Matrix for solving
        b = U.T @ ratings
        
        
        self.movie_features[movie] = np.linalg.solve(A, b)    
        
        if verbose:
            print(self.movie_features[movie][0])
            
    def __call__(self, test_data):
        """
        Predict the rating of a user/movie pair as the dot-product
        of representation vectors of the user and the movie.

        >>> train_data = Dataset(np.array([0, 0, 1, 1, 2]), np.array([1, 2, 3, 4, 5]), np.array([4.0, 1.0, 1.0, 2.0, 1.0]))
        >>> test_data = Dataset(np.array([0, 1, 2, 1]), np.array([1, 2, 2, 0]), np.array([1.0, 2.0, 2.5, 5.0]))
        >>> mean_predictor = MatrixFactorizationPredictor(train_data)
        >>> mean_predictor(test_data)  # the factorization is not yet optimized here
        array([ 2.62654714, -2.89866225,  0.70909287,  5.29901482])
        """
        predictions=[]
        ####################################
        ### ___ Enter your code here ___ ###
        ####################################
        
        for i in range(len(test_data['movie'])) :
            value_m=test_data.movies[i]
            value_u=test_data.users[i]
            pred=np.dot(self.movie_features[value_m,:],self.user_features[value_u,:].T)
            predictions.append(pred)
        return np.array(predictions)
    
def split_dataset(dataset, p_test=0.1, seed=1):
    """
    Split a dataset randomly into a train and a test part

    Inputs:
        dataset: Dataset
        p_test: float
            propability (0 < p_test < 1) for a data point to go into the test set
        seed: integer

    Returns:
        train_dataset: Dataset
        test_dataset: Dataset

    >>> split_dataset(Dataset(np.array([0, 0]), np.array([1, 0]), np.array([2.0, 1.0])), p_test=0)
    (Dataset(movies=array([0, 0]), users=array([1, 0]), ratings=array([2., 1.])), Dataset(movies=array([], dtype=int64), users=array([], dtype=int64), ratings=array([], dtype=float64)))

    >>> split_dataset(Dataset(np.array([0, 0]), np.array([1, 0]), np.array([2.0, 1.0])), p_test=1)
    (Dataset(movies=array([], dtype=int64), users=array([], dtype=int64), ratings=array([], dtype=float64)), Dataset(movies=array([0, 0]), users=array([1, 0]), ratings=array([2., 1.])))
    """
    # use this generator (https://numpy.org/doc/stable/reference/random/index.html)
    # you should use rng.uniform() once inside this function to match the automatic test case
    # Use the random number generator with the specified seed
    rng = np.random.default_rng(seed)

    random_values = rng.uniform(0, 1, len(dataset.movies))

    test_mask = random_values < p_test

    test_data = Dataset(
        movies=dataset.movies[test_mask],
        users=dataset.users[test_mask],
        ratings=dataset.ratings[test_mask]
    )
    
    train_data = Dataset(
        movies=dataset.movies[~test_mask],
        users=dataset.users[~test_mask],
        ratings=dataset.ratings[~test_mask]
    )

    return train_data, test_data